In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tqdm import tqdm
import argparse
import wandb
from os.path import splitext
from os import listdir
import numpy as np
import os
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image
from torchvision.transforms import functional as TF
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import imageio.v2 as imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision import models
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import glob
import wandb
import random
import numpy as np
from model import *
from utils import *
from dataset import *

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [21]:
dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/SO_Generating_Test/dem'
so_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/SO_Generating_Test/so'
rgb_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/SO_Generating_Test/rgb'
pretrained_model_path = '/home/macula/SMATousi/cluster/docker-images/dem2so_more_data/pre_models/B3_rn50_moco_0099_ckpt.pth'



In [22]:
transform = RGB_RasterTransform()
    
dataset = RGB_RasterTilesDataset(dem_dir=dem_dir, so_dir=so_dir, rgb_dir=rgb_dir, transform=transform)

In [23]:
batch_size = 1
learning_rate = 0.0001
epochs = 1
number_of_workers = 0
image_size = 128
val_percent = 0.9

n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=number_of_workers, pin_memory=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True, drop_last=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    

In [14]:
model = RGB_DEM_to_SO(resnet_output_size=(8, 8), 
                            fusion_output_size=(128, 128), 
                            model_choice = "Unet_1", 
                            resnet_saved_model_path=pretrained_model_path,
                            dropout_rate=0.5).to(device)

/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
from torch.optim import Adam
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

In [ ]:
# Run this if the weights are not already downloaded ...


artifact = run.use_artifact('tousi-team/RGB_DEM_2_SO/model_epoch_200:v6', type='model')
artifact_dir = artifact.download()

In [16]:
model.load_state_dict(torch.load('./artifacts/model_epoch_200:v6/model_epoch_200.pth'))

<All keys matched successfully>

In [17]:
def mIOU(label, pred, num_classes=9):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()

#     pred = pred.view(-1)
#     label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
            # print(iou_now)
        iou_list.append(iou_now)
    return iou_list, present_iou_list, np.mean(present_iou_list)

import torch
import numpy as np

def mIOU_with_true_toleramce(label, pred, num_classes=9):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()

#     pred = pred.view(-1)
#     label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        if (sem_class == 1):
            pred_inds_tol = (pred == sem_class - 1) 
        elif (sem_class > 1):
            pred_inds_tol = (pred == sem_class - 1) 
            pred_inds_tol_2 = (pred == sem_class - 2) 
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            if (sem_class == 1):
                intersection_now = (pred_inds[target_inds]).long().sum().item()
                tol_intersection_now = (pred_inds_tol[target_inds]).long().sum().item()
                union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
                iou_now = (float(intersection_now) + float(tol_intersection_now)) / float(union_now)
                present_iou_list.append(iou_now)
            elif (sem_class > 1):
                intersection_now = (pred_inds[target_inds]).long().sum().item()
                tol_intersection_now = (pred_inds_tol[target_inds]).long().sum().item()
                tol2_intersection_now = (pred_inds_tol_2[target_inds]).long().sum().item()
                union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
                iou_now = (float(intersection_now) + float(tol_intersection_now) + float(tol2_intersection_now)) / float(union_now)
                present_iou_list.append(iou_now)
            else:
                intersection_now = (pred_inds[target_inds]).long().sum().item()
                union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
                iou_now = (float(intersection_now)) / float(union_now)
                present_iou_list.append(iou_now)
            # print(iou_now)
        iou_list.append(iou_now)
    return iou_list, present_iou_list, np.mean(present_iou_list)

def mIOU_with_tolerance(label, pred, num_classes=9):
    pred = torch.softmax(pred, dim=1)
    pred = torch.argmax(pred, dim=1)

    # Flatten label and prediction tensors
#     pred = pred.view(-1)
#     label = label.view(-1)

    iou_list = []
    present_iou_list = []

    for sem_class in range(num_classes):
        # Adjusting for tolerance: consider predictions that are one level above or below the target class
        pred_inds = ((pred == sem_class) | (pred == sem_class - 1) & (pred > 0) | (pred == sem_class + 1) & (pred < num_classes - 1))
        target_inds = ((label == sem_class) | (label == sem_class - 1) & (label > 0) | (label == sem_class + 1) & (label < num_classes - 1))

        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')  # Class not present in the label
        else:
            intersection_now = (pred_inds & target_inds).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)

        iou_list.append(iou_now)

    # Compute the mean of the present IoU scores, ignoring NaNs
    valid_iou_scores = [iou for iou in present_iou_list if not np.isnan(iou)]
    return iou_list, present_iou_list, valid_iou_scores, np.mean(valid_iou_scores) if valid_iou_scores else float('nan')

# Note: You'll need to replace 'F.softmax' and 'torch.argmax' with the appropriate calls to your prediction tensor,
# and ensure your label and pred tensors are in the correct shape and format.

def normalize_and_discretize_for_mIoU(label, pred, num_classes=9):
    # Ensure pred and label are torch tensors
    pred = torch.tensor(pred, dtype=torch.float32)
    label = torch.tensor(label, dtype=torch.float32)
    
    # Normalize by dividing by the maximum value in each tensor
    pred_argmax = torch.argmax(pred, dim=1)
    pred_max = pred_argmax.max() if pred_argmax.max() > 0 else 1  # Avoid division by zero
    label_max = label.max() if label.max() > 0 else 1
    pred_normalized = pred_argmax / pred_max
    label_normalized = label / label_max
    
#     print(pred_normalized.shape)
    
#     print(label_normalized.shape)

    # Discretize the normalized values
    pred_discrete = torch.round(pred_normalized * (num_classes - 1)).long()
    label_discrete = torch.round(label_normalized * (num_classes - 1)).long()

    # Flatten the tensors if they're not already, to prepare for mIoU calculation
    pred_flat = pred_discrete.view(-1)
    label_flat = label_discrete.view(-1)

    iou_list = []
    present_iou_list = []

    for sem_class in range(num_classes):
        pred_inds = (pred_flat == sem_class)
        target_inds = (label_flat == sem_class)

        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')  # Class not present in the label
        else:
            intersection_now = (pred_inds & target_inds).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)

        iou_list.append(iou_now)

    # Compute the mean of the present IoU scores, ignoring NaNs
    valid_iou_scores = [iou for iou in present_iou_list if not np.isnan(iou)]
    return present_iou_list, np.mean(valid_iou_scores) if valid_iou_scores else float('nan')



def compute_mIOU_for_region(label_region, pred_region, mIOU_fn, num_classes=9):
    _,_, mean_iou = mIOU_fn(label_region, pred_region, num_classes)
    return mean_iou

def regionalize_and_visualize_mIoU(label, pred, num_classes=9, grid_size=(8, 8), mIOU_fn=mIOU):
    # Assuming the 'label' and 'pred' are [C, H, W] where C is the number of classes
    height, width = label.shape[-2:]
    region_height = height // grid_size[0]
    region_width = width // grid_size[1]

    heatmap = np.zeros(grid_size)
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            label_region = label[:, i*region_height:(i+1)*region_height, j*region_width:(j+1)*region_width]
            pred_region = pred[:, :, i*region_height:(i+1)*region_height, j*region_width:(j+1)*region_width]
#             print(label_region.shape)
#             print(pred_region.shape)
            
            mean_iou = compute_mIOU_for_region(label_region, pred_region, mIOU_fn, num_classes)
            heatmap[i, j] = mean_iou
    
    # Normalize heatmap for visualization
    heatmap_normalized = heatmap
    
    return heatmap
    
    


In [24]:
model.eval()
val_iter = iter(val_loader)

In [30]:
import rasterio

# Assuming you have a file path to one of your input GeoTIFFs
input_file_path = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/SO_Generating_Test/dem/dem_tile_0.tif'

with rasterio.open(input_file_path) as src:
    transform = src.transform
    crs = src.crs
    width = src.width
    height = src.height
    dtype = src.dtypes[0]  # Assuming all bands have the same data type

# Now, you can use this metadata for your output GeoTIFF
out_meta = {
    'driver': 'GTiff',
    'dtype': dtype,  # Use the same data type as the input
    'nodata': None,  # Define a no-data value if applicable
    'width': width,
    'height': height,
    'count': 1,  # or the number of output bands/channels you have
    'crs': crs,
    'transform': transform
}

In [37]:
import time
import rasterio
from rasterio.transform import from_origin

def calculate_average_mIOU(val_iter, model, num_classes=9):
    total_present_iou_list = []
    total_iou = []
    list_of_ious = []
    list_of_ious_tol = []
    
    list_of_ious_tt = []
    
    total_present_iou_list_tol = []
    total_iou_tol = []
    
    start = time.time()

    for idx, batch in enumerate(tqdm(val_iter)):
        dem = batch['DEM'].to(device)
        so = batch['SO'].to(device)
        rgbs = [batch['RGB'][k].to(device) for k in range(6)]
        
        # Assuming your model's prediction step
        val_outputs = model(dem, rgbs)
        
        pred = F.softmax(val_outputs, dim=1)              
        pred = torch.argmax(pred, dim=1).squeeze(1)
#         transform = from_origin(upper_left_x, upper_left_y, pixel_width, pixel_height)  # These values should be defined based on your data
#         out_meta = {
#             'driver': 'GTiff',
#             'dtype': 'float32',  # Adjust based on your output data type
#             'nodata': None,  # Define a no-data value if applicable
#             'width': val_outputs.shape[2],  # Assuming val_outputs is of shape [batch_size, channels, height, width]
#             'height': val_outputs.shape[1],
#             'count': 1,  # Number of bands, adjust if your output is multi-band
#             'crs': '+proj=latlong',  # Define the coordinate reference system
#             'transform': transform
#         }
        with rasterio.open(f'output_{idx}.tif', 'w', **out_meta) as dst:
            dst.write(pred.cpu().detach().numpy().squeeze(), 1)  # Assuming you want to write the first channel/band

        iou_list_tt, present_iou_list_tt, iou_tt = mIOU_with_true_toleramce(so, val_outputs)
        list_of_ious_tt.append(iou_list_tt)
        # Calculate mIOU for the current batch
        iou_list, present_iou_list, iou = mIOU(so, val_outputs, num_classes)
        if not np.isnan(iou):
            total_iou.append(iou)
            
        iou_list_tol, present_iou_list_tol, valid_iou_scores, iou_tol = mIOU_with_tolerance(so, val_outputs, num_classes)
        if not np.isnan(iou_tol):
            total_iou_tol.append(iou_tol)
            
        total_present_iou_list_tol.extend([iou for iou in valid_iou_scores if not np.isnan(iou)])
        list_of_ious.append(iou_list)
        list_of_ious_tol.append(iou_list_tol)
        # Extending the list of present IoUs for all batches
        total_present_iou_list.extend([iou for iou in present_iou_list if not np.isnan(iou)])
    
    # Compute the mean of present IoUs and overall IoU across all batches
    end = time.time()
    
    print("Computing time = ", end - start)
    mean_present_iou = np.mean(total_present_iou_list) if total_present_iou_list else float('nan')
    mean_iou = np.mean(total_iou) if total_iou else float('nan')
    
    mean_present_iou_tol = np.mean(total_present_iou_list_tol) if total_present_iou_list_tol else float('nan')
    mean_iou_tol = np.mean(total_iou_tol) if total_iou_tol else float('nan')
    
    
    return mean_present_iou, mean_iou, mean_present_iou_tol, mean_iou_tol, list_of_ious_tol, list_of_ious, list_of_ious_tt


In [38]:
val_iter = iter(val_loader)
mean_present_iou, mean_iou, mean_present_iou_tol, mean_iou_tol, list_of_ious_tol, list_of_ious, list_of_ious_tt = calculate_average_mIOU(val_iter, model, num_classes=9)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  4.28it/s]

Computing time =  2.1106841564178467
